In [138]:
import pandas as pd
import unicodedata 
import string
from nltk.stem.porter import *
import unicodedata 
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import numpy as np
from nltk.stem.porter import PorterStemmer
import nltk

In [139]:
movies_data = pd.read_table("train.tsv")
movies_data.shape
movies_data["Sentiment"].unique()


array([1, 2, 3, 4, 0])

In [140]:
movie_test = pd.read_table("test.tsv")
movie_test.shape



(66292, 3)

In [141]:
movies_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [142]:
movies_data['Phrase'] = (movies_data['Phrase'].str.lower()
              .str.decode('utf-8')
              .map(lambda x: unicodedata.normalize('NFKD', x))
              .str.encode('ascii', 'ignore'))

movie_test['Phrase'] = (movie_test['Phrase'].str.lower()
              .str.decode('utf-8')
              .map(lambda x: unicodedata.normalize('NFKD', x))
              .str.encode('ascii', 'ignore'))

In [143]:
cols = ["Phrase", "Sentiment"]
movies_data_new =movies_data[cols]

In [144]:
movies_data_new.to_csv("train.csv", index= False)
movie_test.to_csv("test.csv", index= False)

In [145]:
import csv
with open("test.csv", 'r') as file: 
    test = list(csv.reader(file))

print(len(test))

66293


In [146]:
import csv
with open("train.csv", 'r') as file: 
    reviews = list(csv.reader(file))



In [147]:
def get_text(reviews, score):
    # Join together the text in the reviews for a particular tone.
    #We lowercase to avoid "Not" and "not" being seen as different words, for example.
    return " ".join([r[0].lower() for r in reviews if r[1] == str(score)])

In [148]:
negative_text = get_text(reviews, 0)
somewhat_negative_text = get_text(reviews, 1)
neutral_text = get_text(reviews, 2)
somewhat_positive_text = get_text(reviews, 3)
positive_text = get_text(reviews, 4)



In [149]:
def stop_words(text):
    text = ' '.join([word for word in text.split() if word not in stop])
    return text

 

In [ ]:
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in set(string.punctuation)])
    return s



In [150]:
negative_text = stop_words(negative_text)
#negative_text = remove_punctuation(negative_text)

positive_text = stop_words(positive_text)
#positive_text = remove_punctuation (positive_text)

somewhat_negative_text = stop_words(somewhat_negative_text)
#somewhat_negative_text = remove_punctuation(somewhat_negative_text)

somewhat_positive_text = stop_words(somewhat_positive_text)
#somewhat_positive_text = remove_punctuation (somewhat_positive_text)

neutral_text = stop_words(neutral_text)
#neutral_text = remove_punctuation (neutral_text)

In [151]:
with open("positive_text.txt", "w") as text_file:
    text_file.write(positive_text)

In [152]:
with open("negative_text.txt", "w") as text_file:
    text_file.write(negative_text)

In [153]:
with open("somewhat_negative_text.txt", "w") as text_file:
    text_file.write(somewhat_negative_text)

In [154]:
with open("neutral_text.txt", "w") as text_file:
    text_file.write(neutral_text)

In [155]:
with open("somewhat_positive_text.txt", "w") as text_file:
    text_file.write(neutral_text)

LSTM

In [156]:
from __future__ import unicode_literals
import os
import nltk
import pandas as pd
import string
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
np.random.seed(1337)  # For Reproducibility
import multiprocessing
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from utils import *
import glob
from nltk.stem.porter import *
import string
import unicodedata
from joblib import Parallel, delayed
import tqdm

In [231]:
def load_data():
    from sklearn.model_selection import train_test_split
    import io
    sentences_pos = []
    ff = "positive_text.txt"
    with io.open(ff, 'r', encoding='utf8') as f:
        for line in f:
            sentences_pos.append(line)
    sentences_neg = []
    ff = "negative_text.txt"
    with io.open(ff, 'r', encoding='utf8') as f:
        for line in f:
            sentences_neg.append(line)
    
    sentences_smwn = []
    ff = "somewhat_negative_text.txt"
    with io.open(ff, 'r', encoding='utf8') as f:
        for line in f:
            sentences_smwn.append(line)
            
    sentences_smwp = []    
    ff = "somewhat_positive_text.txt"
    with io.open(ff, 'r', encoding='utf8') as f:
        for line in f:
            sentences_smwp.append(line)
            
    sentences_neutral = []        
    ff = "neutral_text.txt"
    with io.open(ff, 'r', encoding='utf8') as f:
        for line in f:
            sentences_neutral.append(line)
            
            
    X = sentences_pos+sentences_neg
    y = [4]*len(sentences_pos)+[0]*len(sentences_neg)+[1]*len(sentences_smwn)+[2]*len(sentences_neutral) + [3]*len(sentences_smwp)
    return X, y

In [232]:
X, y = load_data()
y

[4, 0, 1, 2, 3]

In [234]:
def stemmering_sentences(sentence):
    # Remove punctuation, stopword and then stemmering
    punctuation = set(string.punctuation)
    stemmer = nltk.PorterStemmer()
    # tmp = unicode(sentence, errors='ignore')
    tmp = sentence
    doc = [stemmer.stem(word.lower()) for word in nltk.word_tokenize(tmp) if
           (word not in punctuation) and (word not in nltk.corpus.stopwords.words('english')) and (word != 'br')]
    return doc

def stemmering_sentences_mrd(X, y):
    sentences_stem = Parallel(n_jobs=4)(delayed(stemmering_sentences)(sentence) for sentence in tqdm.tqdm(X, desc="stem"))
    return sentences_stem, y

In [195]:
X, y = stemmering_sentences_mrd(X, y)

stem: 100%|██████████| 2/2 [00:00<00:00,  5.68it/s]


In [222]:
# set parameters:
vocab_dim = 20
maxlen = 100
n_iterations = 1  # ideally more..
# Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. 
n_exposures = 3
window_size = 5
batch_size = 32
n_epoch = 2
input_length = 100
cpu_count = multiprocessing.cpu_count()
cpu_count

8

In [215]:
# combine_train_test_X = terms_by_doc_train + terms_by_doc_test
combine_train_test_X = X

In [225]:
print('Training a Word2vec model...')
model = Word2Vec(size=vocab_dim,
                 min_count=n_exposures,
                 window=window_size,
                 workers=cpu_count,
                 iter=n_iterations)
model.build_vocab(combine_train_test_X)
model.train(combine_train_test_X)
#print len(model.vocab.viewkeys()) -> 11403

Training a Word2vec model...


20000

In [217]:
gensim_dict = Dictionary()
gensim_dict.doc2bow(model.vocab.keys(), allow_update=True)
# gensim_dict.items() returns [(0, u"'surpris"), (1, u'woodi'), (2, u'yellow'),...]
# K+1 aims at avoiding 0 as index.
w2indx = {v: k+1 for k, v in gensim_dict.items()}
w2vec = {word: model[word] for word in w2indx.keys()}
# print len(model["surpris"]) -> 100
print('Setting up Arrays for Keras Embedding Layer...')
n_symbols = len(w2indx) + 1  # adding 1 to account for 0th index
embedding_weights = np.zeros((n_symbols + 1, vocab_dim))
for word, index in w2indx.items():
    embedding_weights[index, :] = w2vec[word]
# print embedding_weights.shape -> (11405, 100)

Setting up Arrays for Keras Embedding Layer...


In [236]:
from sklearn.model_selection import train_test_split
terms_by_doc_train, terms_by_doc_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=58)
X_train = []
for doc in terms_by_doc_train:
    new_txt = []
    for word in doc:
        try:
            new_txt.append(w2indx[word])
        except:
            new_txt.append(0)
    X_train.append(new_txt)
X_test = []
for doc in terms_by_doc_test:
    new_txt = []
    for word in doc:
        try:
            new_txt.append(w2indx[word])
        except:
            new_txt.append(0)
    X_test.append(new_txt)

AttributeError: 'list' object has no attribute 'reshape'